In [1]:
import os
import pickle

import autocnet
import pandas as pd

%pylab inline

root = '/work/users/cneubauer/ctx_trio'

Populating the interactive namespace from numpy and matplotlib


In [2]:
adjacency = {'G03_019262_1888_XN_08N203W.cal.cub':['P02_001949_1883_XN_08N203W.cal.cub', 'B19_017205_1888_XN_08N203W.cal.cub'],
             'P02_001949_1883_XN_08N203W.cal.cub':['B19_017205_1888_XN_08N203W.cal.cub', 'G03_019262_1888_XN_08N203W.cal.cub'],
             'B19_017205_1888_XN_08N203W.cal.cub':['P02_001949_1883_XN_08N203W.cal.cub','G03_019262_1888_XN_08N203W.cal.cub']}

In [3]:
# Create a candidate graph from the adjacency above
cg = autocnet.CandidateGraph.from_adjacency(adjacency, basepath=root)

In [4]:
# Load the features onto each node
# These are the raw extracted features that are 'interesting'
# This also loads the cameras that can be used for reprojection
for i, n in cg.nodes(data='data'):
    n.load_features(os.path.join(root, n['image_name'] + '_kps.h5'), format='hdf')
    with open(os.path.join(root, n['image_name'][:-8] + '.camera'), 'rb') as f:
        cam = pickle.load(f)
        n.camera = cam

In [5]:
# cg.extract_features(extractor_parameters = {'nfeatures': 25})

In [6]:
cg.match()

In [7]:
cg.symmetry_checks()

In [8]:
cg.ratio_checks(clean_keys=['symmetry'])

In [9]:
cg[0][1]['data'].matches

,source_image,source_idx,destination_image,destination_idx,distance
0,0.0,231797.0,1.0,0.0,320.191010
1,0.0,231798.0,1.0,0.0,320.191010
2,0.0,97042.0,1.0,1.0,274.732239
3,0.0,12554.0,1.0,1.0,284.614502
4,0.0,172657.0,1.0,2.0,353.821625
5,0.0,147905.0,1.0,2.0,360.581879
6,0.0,39345.0,1.0,3.0,361.181183
7,0.0,55473.0,1.0,3.0,377.032074
8,0.0,97129.0,1.0,4.0,318.875824
9,0.0,89226.0,1.0,4.0,324.579407


In [ ]:
# cg.compute_fundamental_matrices(clean_keys=['ratio', 'symmetry'])

In [ ]:
# cg.edges[0, 1]['data']['fundamental_matrix']

In [ ]:
# numpy.save( '/work/users/cneubauer/ctx_trio/G03_P02_fundamental.npy' , cg.edges[0, 1]['data']['fundamental_matrix'] )

In [ ]:
stop

In [ ]:
numpy.load( '/work/users/cneubauer/ctx_trio/G03_P02_fundamental.npy' )

In [ ]:
cg.edges[0, 2]['data']['fundamental_matrix']

In [ ]:
# numpy.save( '/work/users/cneubauer/ctx_trio/G03_B19_fundamental.npy' , cg.edges[0, 2]['data']['fundamental_matrix'] )

In [ ]:
numpy.load( '/work/users/cneubauer/ctx_trio/G03_B19_fundamental.npy' )

In [ ]:
cg.edges[1, 2]['data']['fundamental_matrix']

In [ ]:
# numpy.save( '/work/users/cneubauer/ctx_trio/P02_B19_fundamental.npy' , cg.edges[1, 2]['data']['fundamental_matrix'] )

In [ ]:
numpy.load( '/work/users/cneubauer/ctx_trio/P02_B19_fundamental.npy' )

In [ ]:
cg.edges[1,2]['data'].destination

In [11]:
# Load the matches
# Since these matches can from autocnet_server (and a larger project) we remap the source and destination columns to the
#  node ids in this project.
for s, d, e in cg.edges(data='data'):
    source_key = e.source['image_name'][:3]
    destin_key = e.destination['image_name'][:3]
    matches = pd.read_csv(os.path.join(root, '{}_{}_matches.csv'.format(source_key, destin_key)))
    matches['source'] = s
    matches['destination'] = d
    e.matches = matches

In [ ]:
cg.edges[0,1]['data'].plot()

In [ ]:
cg.edges[0, 1]['data'].get_keypoints('source', index=matches['source_idx'])

In [12]:
cg.compute_fundamental_matrices()

/scratch/autocnet/autocnet/transformation/fundamental_matrix.py:314: UserWarning: F computation fell back to 7-point algorithm, not setting F.
  warnings.warn('F computation fell back to 7-point algorithm, not setting F.')
/scratch/autocnet/autocnet/transformation/fundamental_matrix.py:336: UserWarning: Unable to apply MLE.  Not enough correspondences.  Returning with a RANSAC computed F matrix.
  warnings.warn("Unable to apply MLE.  Not enough correspondences.  Returning with a RANSAC computed F matrix.")


In [18]:
cg[0][1]['data'].matches

,Unnamed: 0,id,source,source_idx,destination,destination_idx,lat,lon,geom,source_x,source_y,destination_x,destination_y,valid
0,0,27349,0,192981,1,36039,8.255806,156.860502,01010000a08c7e0e004588823b899b63403d929900f982...,3885.760010,981.765991,3404.513750,6441.123906,True
1,4,27328,0,106672,1,249846,9.065854,156.858770,01010000a08c7e0e006524ab0a7b9b6340c721aca2b721...,4939.462402,8883.077148,3608.247891,6359.811094,True
2,5,27323,0,24593,1,54971,8.534411,156.710969,01010000a08c7e0e00b5474741c096634012f93e599e11...,2654.208008,3875.131592,818.145371,8780.114063,True
3,9,27327,0,160975,1,242873,8.766931,156.494418,01010000a08c7e0e0094ca2045d28f634028b4cb2cab88...,656.461609,6401.264160,3121.315312,13203.950625,True
4,10,27329,0,3689,1,184207,8.187748,156.649140,01010000a08c7e0e00aac13bc0c5946340b7c2ea782060...,1543.436523,571.126343,2672.201719,7847.741094,True
5,11,27333,0,18437,1,48416,8.452987,156.815473,01010000a08c7e0e00a49e1f5b189a63400200e3f5ede7...,3661.822266,2957.137451,3180.672187,8416.235625,True
6,12,27334,0,164542,1,217686,9.043916,156.834229,01010000a08c7e0e00f4c04c01b29a6340e143382f7c16...,4642.079590,8697.544922,4148.713906,14154.512188,True
7,13,27335,0,188810,1,235465,8.807903,156.748070,01010000a08c7e0e008bbb3431f09763403d235375a59d...,3405.508789,6497.782227,2921.959531,11956.275938,True
8,17,27339,0,124116,1,100134,8.414658,156.587860,01010000a08c7e0e00e9ef61c0cf926340a55fd7054ed4...,1189.677979,2855.007324,687.752324,8313.098750,True
9,18,27340,0,193484,1,95540,8.330991,156.831648,01010000a08c7e0e000effe4db9c9a634024cca7a977a9...,3674.577881,1749.249878,3193.456406,7208.303594,True


In [17]:
cg[1][2]['data'].matches

,Unnamed: 0,id,source,source_idx,destination,destination_idx,lat,lon,geom,source_x,source_y,destination_x,destination_y,valid
0,2,26491,2,41418,1,12155,8.331557,156.600594,01010000a08c7e0e00a6ab5a1038936340aee91cdac1a9...,754.123657,7484.470215,3072.454297,1963.866289,True
1,8,26487,2,93999,1,9446,8.301703,156.704458,01010000a08c7e0e00748b93eb8a966340cbbc6ac9789a...,1835.765015,7069.934082,4150.725000,1550.007305,True
2,9,26488,2,39411,1,10202,8.305610,156.640416,01010000a08c7e0e00085251497e946340324b6bf4789c...,1151.415283,7184.391113,3467.459766,1664.297852,True
3,15,26499,2,65116,1,33891,8.720032,156.547389,01010000a08c7e0e000784d73684916340dd69f412a870...,691.410034,11334.522461,3012.363125,5815.006562,True
4,16,26500,2,230329,1,46517,9.003930,156.650194,01010000a08c7e0e000dab0e63ce946340837936100302...,2168.195557,13979.733398,4487.297188,8459.795625,True
5,39,26508,2,97898,1,73656,8.393420,156.720995,01010000a08c7e0e00f1343c63129763406b922c5b6ec9...,2132.468506,7944.500488,4450.112812,2424.552031,True
6,61,26547,2,173808,1,171559,8.286297,156.612128,01010000a08c7e0e0061039b8c96936340926c35879592...,819.425415,7030.128906,3136.437266,1509.579453,True


In [16]:
cg[0][2]['data'].matches

,Unnamed: 0,id,source,source_idx,destination,destination_idx,lat,lon,geom,source_x,source_y,destination_x,destination_y,valid
0,3,26841,2,225920,0,235089,9.362440,156.430689,01010000a08c7e0e00b548c434c88d634068340cca91b9...,760.126221,12284.609375,2568.152969,12218.264687,True
1,4,26842,2,220382,0,219858,9.367159,156.416634,01010000a08c7e0e0066b46a10558d634064fff23ffcbb...,616.222473,12348.048828,2425.657187,12281.315625,True
2,5,26844,2,217883,0,203682,9.464383,156.455625,01010000a08c7e0e008f9f9f7a948e63406fef59a5c3ed...,1158.640747,13249.359375,1892.642734,11937.811875,True
3,7,26846,2,230505,0,246453,9.389877,156.368326,01010000a08c7e0e00d23a2354c98b634063f38ef49dc7...,127.508820,12627.122070,2054.378906,13159.079063,True
4,9,26847,2,218693,0,217917,9.330751,156.509247,01010000a08c7e0e00e9c5e6c04b906340793b003458a9...,1552.820068,11881.565430,3042.583672,13107.296562,True
5,10,26848,2,72091,0,4905,8.230170,156.768664,01010000a08c7e0e002e3b30e698986340d48878d3d875...,2869.854492,841.522278,1943.262500,12560.681562,True
6,11,26849,2,225996,0,235201,9.364555,156.426997,01010000a08c7e0e00a0741ef6a98d634009b5a2e7a6ba...,723.497070,12309.653320,3361.284844,11816.145000,True
7,12,26850,2,217738,0,203531,9.451786,156.371382,01010000a08c7e0e00c055c45be28b634042ce9b7d50e7...,242.727264,13226.710938,4708.027656,779.158555,True
8,13,26851,2,231701,0,247830,9.466197,156.469770,01010000a08c7e0e00872c735b088f63409118b96eb1ee...,1310.947266,13249.793945,2530.534453,12242.491250,True
9,23,26862,2,230344,0,246283,9.382636,156.376181,01010000a08c7e0e003a0c34ac098c634033d858e4e8c3...,202.548752,12547.117188,2985.201406,12581.338125,True
